In [0]:
from functools import total_ordering
import random
import math

In [0]:
class Heap(list):
    """继承 python 内置的 list 类型，并添加堆的属性"""
    def __init__(self, array, heap_size=None, d=2):
        list.__init__(self, array)
        self.heap_size = len(array) if heap_size is None else heap_size
        self.d = d  # d 叉堆
        
    def __str__(self):
      """如果是二叉堆，则返回逆时针旋转后的字符串"""
      def recurse(index, level):
        s = ""
        if index < self.heap_size:
          s += recurse(right(index), level + 1)
          s += "|  " * level
          s += str(self[index]) + "\n"
          s += recurse(left(index), level + 1) 
        return s
      
      if self.d == 2:
        return recurse(0, 0)
      else:
        return str(self[:self.heap_size])

def print_matrix(A):
  """打印二维数组"""
  print("[ ")
  for i in A:
    for j in i:
      print('{:^3}'.format(j), end=', ')
    print('\n')
  print("]")

## 6.1 堆 

- 堆是一个数组，可以被看成一个近似的完全二叉树（除了最底层，该树应该是完全充满的）
- 示意图  
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20191211201233.png width = 600>
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200111173051.png width=800>
- $A\left[ {1 \ldots A.length} \right]$ 中可能都含有元素，但只有 $A\left[ {1 \ldots A.heap\_size} \right]$ 中存放的是有效元素

#### **获取给定节点的父节点，左孩子结点和右孩子结点** 


- 常用编程语言的下标从 0 开始，除了代码部分，则默认数组下标从 1 开始
- 可通过左移或右移操作来加快计算速度
- python 的移位操作优先级较低，注意加括号以避免出错

In [0]:
def parent(i):
    return (i-1) >> 1

def left(i):
    return (i << 1) + 1

def right(i):
    return (i << 1) + 2

In [0]:
i = 3
print("the parent of i is {}, the left child of i is {}, the right child of i is {}". format(parent(i), left(i), right(i)))

the parent of i is 1, the left child of i is 7, the right child of i is 8


#### 最大堆与最小堆

- 最大堆的性质
    - 对于除了根节点以外的所有节点 $i$ 都要满足：$$A\left[ {PARENT\left( i \right)} \right] \ge A\left[ i \right]$$
    
    - 即最大堆中的最大元素存放在根节点中，且在任一子树中，这一性质也成立


- 最小堆的性质
    - 对于除了根节点以外的所有节点 $i$ 都要满足：$$A\left[ {PARENT\left( i \right)} \right] \le A\left[ i \right]$$
    
    - 即最小堆中的最小元素存放在根节点中，且在任一子树中，这一性质也成立

#### 堆的应用

- 排序
- 优先队列

#### 堆的性质

- 包含 $n$ 个元素的堆高度为 $\Theta(lg(n))$
- 堆结构上的一些基本操作至多与堆的高度成正比，即时间复杂度为 $O(lg(n))$
- 最大堆的最小元素位于叶节点中
- 已排好序的数组是一个最小堆
- 当用数组表示储存 $n$ 个元素的堆时，叶结点的下标分别为 $\left\lfloor {{n \over 2}} \right\rfloor  + 1,\left\lfloor {{n \over 2}} \right\rfloor  + 2, \cdots ,n$, 即叶节点的数目为 $\left\lceil {{n \over 2}} \right\rceil$
- 当只含有一个元素时，堆的高度为 0

## 6.2 维护堆的性质

- 在调用 MAX-HEAPIFY 假设：
    - 根结点为 LEFT($i$) 和 RIGHT($i$) 的二叉树都是最大堆
    - $A[i]$ 可能小于其孩子节点

### 代码实现

In [0]:
def max_heapify(A, i):
    l = left(i)
    r = right(i)
    largest = i
    if l < A.heap_size and A[l] > A[largest]:
        largest = l
    if r < A.heap_size and A[r] > A[largest]:
        largest = r
    if largest != i:
        A[i], A[largest] = A[largest], A[i]
        max_heapify(A, largest)

In [0]:
A = Heap([16, 4, 10, 14, 7, 9, 3, 2, 8, 1])
print("MAX-HEAPIFY前：\n{}".format(A))
max_heapify(A, 1)
print("MAX-HEAPIFY后：\n{}".format(A))

MAX-HEAPIFY前：
|  |  3
|  10
|  |  9
16
|  |  7
|  |  |  1
|  4
|  |  |  8
|  |  14
|  |  |  2

MAX-HEAPIFY后：
|  |  3
|  10
|  |  9
16
|  |  7
|  |  |  1
|  14
|  |  |  4
|  |  8
|  |  |  2



### 程序执行示意图

<img src="https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20191211201929.png" width = 600>

###  复杂度分析

- 调用 $A[i], A[LEFT(i)], A[RIGHT(i)]$ 的时间代价为 $\Theta(1)$
- 子树的规模最大为 $ 2n/3 $
  - 发生在最底层恰好半满的时候
      - 设树高为 $h$，则除最底层个，包含的元素数为：
      $$1 + 2 + {2^2} +  \cdots  + {2^{h - 1}} = {{{2^h} - 1} \over {2 - 1}} = {2^h} - 1$$
      - 左右子树各占 $(2^h-1)/2 = 2^{h-1} - 1/2$
      - 最后一层半满时，元素数为 $2^{h-1}$
      - 最大子树的规模为：
      $${{{2^{h - 1}} + {{\rm{2}}^{h - 1}} - 1/2} \over {{2^{h - 1}} + {2^h} - 1}} \approx {2 \over 3}$$
- 运行时间的递推式为：$$T(n) \le T(2n/3) + \Theta(1)$$

由主定理可得，运行时间为 $O\left( {\lg n} \right)$，即 $O(h)$

###  练习

####  6.2-2 MINI-HEAPIFY

In [0]:
def mini_heapify(A, i):
    l = left(i)
    r = right(i)
    smallest = i
    if l < A.heap_size and A[l] < A[smallest]:
        smallest = l
    if r < A.heap_size and A[r] < A[smallest]:
        smallest = r
    if smallest != i:
        A[smallest], A[i] = A[i], A[smallest]
        mini_heapify(A, smallest)

In [0]:
A = Heap([1, 10, 3, 4, 5, 6, 7, 8, 9, 10])
print("MINI-HEAPIFY前：\n{}".format(A))
mini_heapify(A, 1)
print("MINI-HEAPIFY后：\n{}".format(A))

MINI-HEAPIFY前：
|  |  7
|  3
|  |  6
1
|  |  5
|  |  |  10
|  10
|  |  |  9
|  |  4
|  |  |  8

MINI-HEAPIFY后：
|  |  7
|  3
|  |  6
1
|  |  5
|  |  |  10
|  4
|  |  |  9
|  |  8
|  |  |  10



### 6.2-5 用循环控制结构重写 MAX-HEAPIFY

In [0]:
def max_heapify_loop(A, i):
    while True:
        l = left(i)
        r = right(i)
        largest = i
        if l < A.heap_size and A[l] > A[largest]:
            largest = l
        if r < A.heap_size and A[r] > A[largest]:
            largest = r
        if largest != i:
            A[i], A[largest] = A[largest], A[i]
            i = largest
        else:
            break

In [0]:
A = Heap([16, 4, 10, 14, 7, 9, 3, 2, 8, 1])
print("MAX-HEAPIFY前：\n{}".format(A))
max_heapify_loop(A, 1)
print("MAX-HEAPIFY后：\n{}".format(A))

MAX-HEAPIFY前：
|  |  3
|  10
|  |  9
16
|  |  7
|  |  |  1
|  4
|  |  |  8
|  |  14
|  |  |  2

MAX-HEAPIFY后：
|  |  3
|  10
|  |  9
16
|  |  7
|  |  |  1
|  14
|  |  |  4
|  |  8
|  |  |  2



## 6.3 建堆

- 子数组 $A\left( {\left\lfloor {n/2} \right\rfloor  + 1 \ldots n} \right)$ 中的元素都是树的叶节点
- 每个叶节点都可以看成只包含一个元素的堆
- 因此， BUILD-MAX-HEAP 对树中的其他节点都调用一次 MAX-HEAPIFY 即可

###  代码实现

In [0]:
def build_max_heap(A):
    A.heap_size = len(A)
    for i in reversed(range(len(A)//2)):
        max_heapify(A, i)

In [0]:
A = Heap([4, 1, 3, 2, 16, 9, 10, 14, 8, 7])
print("建最大堆前：\n{}".format(A))
build_max_heap(A)
print("建最大堆后：\n{}".format(A))

建最大堆前：
|  |  10
|  3
|  |  9
4
|  |  16
|  |  |  7
|  1
|  |  |  8
|  |  2
|  |  |  14

建最大堆后：
|  |  3
|  10
|  |  9
16
|  |  7
|  |  |  1
|  14
|  |  |  4
|  |  8
|  |  |  2



###  示意图

<img src="https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20191211211919.png" width=600>

### 复杂度分析

- 高度为 $h$ 的子树至多有 $\lceil n/2^{h+1}\rceil$ 个(练习 6.3-3)
- 高度为 $h$ 的子树的根节点调用 MAX-HEAPIFY 的时间复杂度为 $O(h)$
- 总的时间复杂度为：
$$\sum\limits_{h = 0}^{\left\lfloor {\lg n} \right\rfloor } {\left\lceil {{n \over {{2^{h + 1}}}}} \right\rceil } O\left( h \right) = O\left( {n\sum\limits_{h = 0}^{\left\lfloor {\lg n} \right\rfloor } {{h \over {{2^h}}}} } \right)$$
  - 无限几何级数如下
    - $$\sum_{k=0}^{\infty} x^{k}=\frac{1}{1-x}$$
  - 对此式两边求微分，可得
    - $$\sum_{k=0}^{\infty} k x^{k}=\frac{x}{(1-x)^{2}}$$

可得：
$$O\left( {n\sum\limits_{h = 0}^{\left\lfloor {\lg n} \right\rfloor } {{h \over {{2^h}}}} } \right) = O\left( {n \times {{{1 \over 2}} \over {{{\left( {1 - {1 \over 2}} \right)}^2}}}} \right) = O\left( n \right)$$
由此可得，建堆的时间复杂度为 $O(n)$

### 练习

#### 6.3 -3
证明对于一个包含 $n$ 个元素的堆，至多有 $\lceil n/2^{h+1}\rceil个子树$

1. 包含 $n$ 个元素的堆,其叶子节点的数目为 $\lceil n/2 \rceil$
2. 对于堆的子树而言，其叶子节点即为堆的叶子节点，而且高度为 $h$ 的各个子树的叶子节点不会有重复。假设高度为 $h$ 的子树一共有 $k$ 个，则有 $ k - 1 $ 个子树的叶子节点为 $2^h$ 而一个子树的叶子节点在 $[1, 2^h]$ 之间, 设为 $x$  
由 1，2 可得：
$$(k-1)2^h + x \le k * 2^h \le \lceil n/2 \rceil$$
即：
$$k \le {{\lceil n/2 \rceil} \over 2^h} \le \left\lceil {{{\left\lceil {{n \over 2}} \right\rceil } \over {{{\rm{2}}^h}}}} \right\rceil  $$
由书中公式 (3.4) 可得：
$$k \le \left\lceil {{n \over {{2^{h + 1}}}}} \right\rceil $$
证毕

## 6.4 堆排序算法

- 基本思路  
  1. 先根据数组建最大堆
  2. 从最大堆顶取出最大的元素与堆的最后一个元素交换，然后将堆的长度减 1，并维护堆的性质
  3. 重复 2 中的步骤，直到最大堆中的元素数目等于 1

### 代码实现

In [0]:
def heapsort(A):
    build_max_heap(A)
    while A.heap_size > 1:
        A[0], A[A.heap_size - 1] = A[A.heap_size - 1], A[0]
        A.heap_size = A.heap_size - 1
        max_heapify(A, 0)

In [0]:
A = Heap([4, 1, 3, 2, 16, 9, 10, 14, 8, 7])
heapsort(A)
print(super(Heap,A).__str__())

[1, 2, 3, 4, 7, 8, 9, 10, 14, 16]


### 代码执行过程

<img src="https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20191212204631.png" width=600>

### 复杂度分析

- BUILD-MAX-HEAP 的时间复杂度为 $O(n)$
- MAX-HEAPIFY 的时间复杂度为 $O(lgn)$，会被调用 $n-1$ 次
- 整体的时间复杂变为 $O(nlg(n))$

## 6.5 优先队列

- 优先队列是一种用来维护由一组元素组成的集合 $S$ 的数据结构，其中每种元素都有一个相关的值，称为关键字(key)

### 最大优先队列

- 支持如下操作  
  - INSERT(S, x): 将元素 $x$ 插入到集合 $S$ 中
  - MAXIMUN(S,x): 返回 $S$ 中具有最大键值的元素
  - EXTRACT-MAX(S): 去掉并返回 $S$ 中具有最大键值的元素
  - INCREASE-KEY(S, x, k)： 将元素 $x$ 的关键值增加到 $k$，要求 $k$ 不小于 $x$ 原关键字的值
- 应用实例
  - 计算机中的作业调度，可以随时使用 INSERT 将新的作业插入进来

### 最小优先队列

- 包括 INSERT, MINIMUN, EXTRACT_MIN 和 DECREASE-KEY 操作
- 应用实例
  - 基于事件驱动的模拟器
    - 事件必须按照发生的时间顺序进行模拟，因为某一事件的模拟结果可能触发对其它事件的模拟

### 最大优先队列常用操作的代码实现
- 一般需要在堆中储存对应对象的句柄

#### HEAP-MAXIMUN

In [0]:
def heap_maximun(A):
  if A.heap_size < 1:
    raise Exception("Heap underflow")
  return A[0]

In [0]:
A = Heap([16, 14, 10, 8, 7, 9, 3, 2, 4, 1])
heap_maximun(A)

16

#### HEAP-EXTRACT-MAX

In [0]:
def heap_extract_max(A):
  if A.heap_size < 1:
    raise Exception("Heap underflow")
  max = A[0]
  A[0] = A[A.heap_size-1]
  A.heap_size -= 1
  max_heapify(A, 0)
  return max

In [0]:
A = Heap([16, 14, 10, 8, 7, 9, 3, 2, 4, 1])
print(A)
print(heap_extract_max(A))
print()
print(A)

|  |  3
|  10
|  |  9
16
|  |  7
|  |  |  1
|  14
|  |  |  4
|  |  8
|  |  |  2

16

|  |  3
|  10
|  |  9
14
|  |  7
|  8
|  |  |  1
|  |  4
|  |  |  2



#### HEAP-INCREASE-KEY

In [0]:
def heap_increase_key(A, i, key):
  if not(0<= i <A.heap_size):
    raise Exception("index out of range!")
  if key < A[i]:
    raise Exception("new key is smaller than current key!")
  A[i] = key
  while i > 0 and A[parent(i)] < A[i]:
    A[i], A[parent(i)] = A[parent(i)], A[i]
    i = parent(i)

In [0]:
A = Heap([16, 14, 10, 8, 7, 9, 3, 2, 4, 1])
print(A)
heap_increase_key(A, 8, 15)
print(A)

|  |  3
|  10
|  |  9
16
|  |  7
|  |  |  1
|  14
|  |  |  4
|  |  8
|  |  |  2

|  |  3
|  10
|  |  9
16
|  |  7
|  |  |  1
|  15
|  |  |  8
|  |  14
|  |  |  2



- 执行过程示意图  
<img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20191213110736.png width=600>

#### MAX-HEAP-INSERT
- 借助 HEAP-INCREASE-KEY 来实现


In [0]:
def max_heap_insert(A, key):
  A.heap_size = A.heap_size + 1
  if A.heap_size > len(A):
    A.append(-float('inf'))
  else:
    A[A.heap_size-1] = -float('inf')
  heap_increase_key(A, A.heap_size-1, key)

In [0]:
A = Heap([16, 14, 10, 8, 7, 9, 3, 2, 4, 1])
print(A)
max_heap_insert(A, 15)
print(A)

|  |  3
|  10
|  |  9
16
|  |  7
|  |  |  1
|  14
|  |  |  4
|  |  8
|  |  |  2

|  |  3
|  10
|  |  9
16
|  |  |  7
|  |  14
|  |  |  1
|  15
|  |  |  4
|  |  8
|  |  |  2



### 练习题

#### 6.5-3 最小优先队列常用的操作

##### HEAP-MINIMUM

In [0]:
def heap_minimun(A):
  if A.heap_size < 1:
    raise Exception("Heap underflow")
  return A[0]

##### HEAP-EXTRACT-MIN

In [0]:
def heap_extract_min(A):
  if A.heap_size < 1:
    raise Exception("heap underflow")
  mini = A[0]
  A.heap_size -= 1
  A[0] = A[A.heap_size]
  mini_heapify(A, 0)
  return mini

In [0]:
A = Heap([1, 2, 3, 4, 5, 6, 7, 8, 9])
print(A)
print(heap_extract_min(A), end="\n\n")
print(A)

|  |  7
|  3
|  |  6
1
|  |  5
|  2
|  |  |  9
|  |  4
|  |  |  8

1

|  |  7
|  3
|  |  6
2
|  |  5
|  4
|  |  8
|  |  |  9



##### HEAP-DECREASE-KEY

In [0]:
def heap_decrease_key(A, i, key):
  if not(0 <= i < A.heap_size):
    raise Exception('List out of range!')
  if A[i] < key:
    raise Exception('new key is bigger than current key!')
  A[i] = key
  while i > 0 and A[parent(i)] > key:
    A[i] = A[parent(i)]
    i = parent(i)
  A[i] = key

In [0]:
A = Heap([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
print(A)
heap_decrease_key(A, 4, 0)
print(A)

|  |  7
|  3
|  |  6
1
|  |  5
|  |  |  10
|  2
|  |  |  9
|  |  4
|  |  |  8

|  |  7
|  3
|  |  6
0
|  |  2
|  |  |  10
|  1
|  |  |  9
|  |  4
|  |  |  8



##### MINI-HEAP-INSERT

In [0]:
def mini_heap_insert(A, key):
  A.heap_size += 1
  if A.heap_size > len(A):
    A.append(float('inf'))
  else:
    A[A.heap_size - 1] = float('inf')
  heap_decrease_key(A, A.heap_size-1, key)

In [0]:
A = Heap([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
print(A)
mini_heap_insert(A, 2)
print(A)

|  |  7
|  3
|  |  6
1
|  |  5
|  |  |  10
|  2
|  |  |  9
|  |  4
|  |  |  8

|  |  7
|  3
|  |  6
1
|  |  |  5
|  |  2
|  |  |  10
|  2
|  |  |  9
|  |  4
|  |  |  8



#### 6.5-6 一次赋值完成 HEAP-INCREASE-KEY 操作

In [0]:
def heap_increase_key_one_assignment(A, i, key):
  if not(0<= i <A.heap_size):
    raise Exception("index out of range!")
  if key < A[i]:
    raise Exception("new key is smaller than current key!")
  A[i] = key
  while i > 0 and A[parent(i)] < key:
    A[i] = A[parent(i)] 
    i = parent(i)
  A[i] = key

In [0]:
A = Heap([16, 14, 10, 8, 7, 9, 3, 2, 4, 1])
print(A)
heap_increase_key_one_assignment(A, 8, 15)
print(A)

|  |  3
|  10
|  |  9
16
|  |  7
|  |  |  1
|  14
|  |  |  4
|  |  8
|  |  |  2

|  |  3
|  10
|  |  9
16
|  |  7
|  |  |  1
|  15
|  |  |  8
|  |  14
|  |  |  2



#### 6.5-8 HEAP-DELETE(A, i)

##### 最大堆

In [0]:
def max_heap_delete(A, i):
  if not(0<= i <=A.heap_size):
    raise Exception('index out of range')
  A.heap_size -= 1
  A[i] = A[A.heap_size]
  max_heapify(A, i)

In [0]:
A = Heap([10, 9, 8, 7, 6, 5, 4, 3, 2, 1])
print(A)
max_heap_delete(A, 1)
print(A)

|  |  4
|  8
|  |  5
10
|  |  6
|  |  |  1
|  9
|  |  |  2
|  |  7
|  |  |  3

|  |  4
|  8
|  |  5
10
|  |  6
|  7
|  |  |  2
|  |  3
|  |  |  1



##### 最小堆

In [0]:
def mini_heap_delete(A, i):
  if not(0<= i <=A.heap_size):
    raise Exception('index out of range')
  A.heap_size -= 1
  A[i] = A[A.heap_size]
  mini_heapify(A, i)

In [0]:
A = Heap([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
print(A)
mini_heap_delete(A, 1)
print(A)

|  |  7
|  3
|  |  6
1
|  |  5
|  |  |  10
|  2
|  |  |  9
|  |  4
|  |  |  8

|  |  7
|  3
|  |  6
1
|  |  5
|  4
|  |  |  9
|  |  8
|  |  |  10



#### 6.5-9 在 $O(nlg(k))$ 内归并 $k$ 个有序链表

##### 基本思路

1. 将 $k$ 个列表的第一个元素取出，建成最小堆，复杂度为 $O(k)$，此堆最多有 $k$ 个元素
2. 借助 HEAP-EXTRACT-MIN 弹出堆中最小的元素，此最小元素为第 i 小的元素（i 为第 2 步执行的次数），并将此最小元素对应的链表的下一个元素（可能为第 i + 1 个最小的元素），借助 MINI-HEAP-INSERT 插入到堆中. HEAP-EXTRACT-MIN 和 MINI-HEAP-INSERT 的时间复杂度均为 $O(lg(k))$
3. 重复 2 中的操作，直到遍历完所有的链表，共会调用第二步 $n$ 次  
综上，总的时间复杂度为 $O(nlg(k))$

####  代码实现

- 自定义 python 链表类

In [0]:
@total_ordering
class LinkedNode():
    def __init__(self, value, next=None):
        self.value = value
        self.next = next
        
    def __eq__(self, other):
      try:
        return self.value == other.value
      except AttributeError:
        return self.value == other
    
    def __lt__(self, other):
      try:
        return self.value < other.value
      except AttributeError:
        return self.value < other

- BUILD_MINI_HEAP

In [0]:
def build_mini_heap(A):
    for i in reversed(range(A.heap_size // 2)):
        mini_heapify(A, i)

- MERGE_SORTED_LINKED_LIST

In [0]:
def merge_sorted_linked_list(linked_lists):
    """
    合并多个有序链表，返回结果会打乱原有链表的顺序
    
    Args:
        linked_lists [list]: 多个有序的链表组成的列表
    Returns:
        sorted_linked_list [LinkedNode]: 排序后的链表
    """
    A = Heap(linked_lists)
    build_mini_heap(A)
    cur = first = LinkedNode(-float('inf'))
    while A.heap_size > 0:
        cur.next = heap_extract_min(A)
        cur = cur.next
        if cur.next is not None:
            mini_heap_insert(A, cur.next)
    return first.next

- 测试程序

In [0]:
k = 3
linked_lists = [LinkedNode(random.randint(1, 100)) for i in range(k)]
for linked_list in linked_lists:
  cur = linked_list
  for i in range(random.randint(1, 6)):
    cur.next = LinkedNode(cur.value + random.randint(0, 10))
    cur = cur.next

#  打印生成的链表
for linked_list in linked_lists:
  cur = linked_list
  while cur is not None:
    print(cur.value, end=" -> ")
    cur = cur.next
  print("None" + "\n")

12 -> 15 -> 23 -> 25 -> None

49 -> 58 -> 63 -> None

91 -> 97 -> 106 -> 109 -> 119 -> 120 -> 128 -> None



In [0]:
res = merge_sorted_linked_list(linked_lists)
cur = res
while cur is not None:
  print(cur.value, end=" -> ")
  cur = cur.next
print('None')

12 -> 15 -> 23 -> 25 -> 49 -> 58 -> 63 -> 91 -> 97 -> 106 -> 109 -> 119 -> 120 -> 128 -> None


## 思考题

### 6-2 对 $d$ 叉堆的分析

#### a.  
如何在一个数组中表示 $d$ 叉堆

假设数组下标从 0 开始，则 $d$ 叉堆可表示为如下形式:
$$\left[ 0\quad {\overbrace {1 \quad \cdots \quad d}^{0}} \quad 
{\overbrace {d+1 \quad \cdots \quad 2d}^{1}} \quad \cdots \quad
{\overbrace {(n-1)*d-1 \quad \cdots \quad x}^{n}} \right]\quad x \le nd$$
其中大括号上的数字为大括号中下标的父节点。  
则下标为 $i$ 的元素其父节点为 $\lfloor (i-1)/d\rfloor$，子节点为 $d*i+k,\quad k \in [1, d]$，同时需要确保子节点的下标不超过 $heap\_size$  
可以用以下代码来表示下标为 $i$ 的元素的父节点和子节点

In [0]:
def d_parent(i, d):
  return (i-1) // d
  
def children(i, d):
  """返回子节点所组成的迭代器, 实际使用时需要判定是否超过 hepa_size"""
  return range(d*i+1, (i+1)*d+1)

In [0]:
list(children(3, 4))

[13, 14, 15, 16]

In [0]:
d_parent(3, 4)

0

#### b.  
包含 $n$ 个元素的 $d$ 叉堆的高度为 $log_d(n)$

#### c.  
EXTRACT-MAX 在 $d$ 叉堆上的时间复杂度  



时间复杂度为 $O(d log_d(n))$，其中 EXTRACT-MAX 可以借助 D-MAX-HEAPFIY 来实现

In [0]:
def d_max_heapify(A, i):
  max = A[i]
  largest = i
  for j in children(i, A.d):
    if j >= A.heap_size:
      break
    else:
      if A[j] > max:
        max = A[j]
        largest = j
  if largest != i:
    A[i], A[largest] = A[largest], A[i]
    d_max_heapify(A, largest)


In [0]:
def d_extract_max(A):
  if A.heap_size < 1:
    raise Exception("heap underflow")
  max = A[0]
  A.heap_size -= 1
  A[0] = A[A.heap_size]
  d_max_heapify(A, 0)
  return max

In [0]:
A = Heap(list(reversed(range(1, 11))), d=4)
print(A)
print(d_extract_max(A))
print(A)

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
10
[9, 5, 8, 7, 6, 1, 4, 3, 2]


In [0]:
A = Heap(list(reversed(range(1, 11))), d=2)
print(A)
print(d_extract_max(A))
print(A)

|  |  4
|  8
|  |  5
10
|  |  6
|  |  |  1
|  9
|  |  |  2
|  |  7
|  |  |  3

10
|  |  4
|  8
|  |  5
9
|  |  6
|  7
|  |  |  2
|  |  3
|  |  |  1



#### e.  
INCREASE-KEY(A, i, k) 的有效实现及时间复杂度  



时间复杂度为 $O(log_d(n))$


In [0]:
def d_increase_key(A, i, key):
  if key < A[i]:
    raise Exception("new value is smaller than current value")
  while i > 0 and A[d_parent(i, A.d)] < key:
    A[i] = A[d_parent(i, A.d)]
    i = d_parent(i, A.d)
  A[i] = key


In [0]:
A = Heap(list(reversed(range(1, 11))), d=4)
print(A)
d_increase_key(A, 6, 100)
print(A)

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
[100, 10, 8, 7, 6, 5, 9, 3, 2, 1]


#### d.  
INSERT 在 d 叉堆上的实现及时间复杂度

可借助 INCREASE_KEY 来实现，时间复杂度为 $O(log_d(n))$

In [0]:
def d_max_heap_insert(A, key):
  A.heap_size += 1
  if A.heap_size > len(A):
    A.append(-float('inf'))
  A[A.heap_size-1] = -float('inf')
  d_increase_key(A, A.heap_size-1, key)

In [0]:
A = Heap(list(reversed(range(1, 11))), d=4)
print(A)
d_max_heap_insert(A,100)
print(A)

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
[100, 9, 10, 7, 6, 5, 4, 3, 2, 1, 8]


### 6-3 Young 氏矩阵

#### b  
对于一个 $m \times n$ 的 Young 氏矩阵 $Y$， $Y[1,1]$ 为其最小的元素，而 $Y[m,n]$ 为其最大的元素

#### C  
复杂度为 $O(m+n)$ 的 EXTRACT-MIN 算法

基本思路： $Y[1, 1]$ 为最小的元素，可将其赋值为 $\infty$ 后，借助 MINI-HEAPIFY-DOWN 来创建新的 Young 氏矩阵  
其中 MINI-HEAPIFY-DOWN(Y, i, j) 调用的前提条件:
- $Y[1 \rightarrow i, 1 \rightarrow j] 符合 Young 氏矩阵的规则$
- $Y[(1 \rightarrow i-1, i + 1 \rightarrow m), 
(1 \rightarrow j-1, j+1 \rightarrow n)] 符合 Young 氏矩阵的规则$

##### MINI-HEAPIFY-DOWN

In [0]:
def mini_heapify_down(A, i, j):
  m, n= len(A), len(A[0])
  mini_i, mini_j = i, j
  if i + 1 < m and A[mini_i][mini_j] > A[i+1][j]:
    mini_i, mini_j = i + 1, j
  if j + 1 < n and A[mini_i][mini_j] > A[i][j+1]:
    mini_i, mini_j = i, j + 1
  if mini_i != i or mini_j != j:
    A[i][j], A[mini_i][mini_j] = A[mini_i][mini_j], A[i][j]
    mini_heapify_down(A, mini_i, mini_j)

##### EXTRACT-MIN

In [0]:
def young_extract_min(A):
  mini = A[0][0]
  A[0][0] = float('inf')
  mini_heapify_down(A, 0, 0)
  return mini

##### TEST

In [0]:
A = [[i + j for i in range(4)] for j in range(4)]
print("原始数组：")
for i in A:
  for j in i:
   print(j, end=', ')
  print('\n')

print("第一次调用:")
print(young_extract_min(A))
print_matrix(A)
print("第二次调用：")
print(young_extract_min(A))
print_matrix(A)


原始数组：
0, 1, 2, 3, 

1, 2, 3, 4, 

2, 3, 4, 5, 

3, 4, 5, 6, 

第一次调用:
0
[ 
 1 ,  1 ,  2 ,  3 , 

 2 ,  2 ,  3 ,  4 , 

 3 ,  3 ,  4 ,  5 , 

 4 ,  5 ,  6 , inf, 

]
第二次调用：
1
[ 
 1 ,  2 ,  2 ,  3 , 

 2 ,  3 ,  3 ,  4 , 

 3 ,  4 ,  5 , inf, 

 4 ,  5 ,  6 , inf, 

]


##### 复杂度分析

每调用 MINI-HAPIFY-DOWN 一次，问题的规模减 1，则可得递推式：
$$ T(p) = T(p-1) + \Theta(1)$$
由递推式可得，运行时间为 $O(p) = O(m+n)$

#### d.  
在 $O(m+n)$ 的时间内，将一个新元素插入到末满的 $ m \times n $ 的 Young 氏矩阵中

基本思路：  
- 由于数组末满，则 $Y[m,n] = \infty$, 可借助 MINI-HEAPIFY-UP 将 新元素插入到 $Y[m, n]$ 中
- MINI-HEAPIFY-DOWN(Y, i, j) 调用的前提条件：
  - $Y[1 \rightarrow i-1, 1 \rightarrow j-1] 符合 Young 氏矩阵的规则$
  - $Y[(1 \rightarrow i-1, i + 1 \rightarrow m), 
(1 \rightarrow j-1, j+1 \rightarrow n)]  符合 Young 氏矩阵的规则$

##### MINI-HEAPIFY-UP

In [0]:
def mini_heapify_up(A, i, j):
  max_i, max_j = i, j
  if i - 1 >= 0 and A[i-1][j] > A[max_i][max_j]:
    max_i, max_j = i-1, j
  if j - 1 >= 0 and A[i][j-1] > A[max_i][max_j]:
    max_i, max_j = i, j - 1
  if max_i != i or max_j != j:
    A[max_i][max_j], A[i][j] = A[i][j], A[max_i][max_j]
    mini_heapify_up(A, max_i, max_j)

##### YOUNG-INSERT

In [0]:
def young_insert(A, key):
  m, n = len(A), len(A[0])
  if A[m-1][n-1] != float('inf'):
    raise Exception("matrix is full")
  A[m-1][n-1] = key
  mini_heapify_up(A, m-1, n-1)

##### TEST

In [0]:
A = [[float('inf') for i in range(3)] for j in range(3) ]
A[0][0] = 1
A[0][1] = 2
print_matrix(A)
for i in range(3):
  key = random.randint(1, 99)
  print("insert {}".format(key))
  young_insert(A, key)
  print_matrix(A)

[ 
 1 ,  2 , inf, 

inf, inf, inf, 

inf, inf, inf, 

]
insert 15
[ 
 1 ,  2 , 15 , 

inf, inf, inf, 

inf, inf, inf, 

]
insert 8
[ 
 1 ,  2 , 15 , 

 8 , inf, inf, 

inf, inf, inf, 

]
insert 76
[ 
 1 ,  2 , 15 , 

 8 , 76 , inf, 

inf, inf, inf, 

]


##### 复杂度分析

每调用 MINI-HAPIFY-UP 一次，问题的规模减 1，则可得递推式：
$$ T(p) = T(p-1) + \Theta(1)$$
由递推式可得，运行时间为 $O(p) = O(m+n)$

#### e  
利用 Young 氏数组在 $O(n^3)$ 时间内将 $n^2$ 个数进行排序

基本思路：
1. 借助 YOUNG-INSERT 将 $n^2$ 个数初始化为 $n \times n$ 的 YOUNG 氏矩阵，YOUNG-INSERT 的复杂度为
$O(2n) = O(n)$，需调用 $n^2$ 次，共需用时 $O(n^3)$
2. 通过 EXTRACT_MIN 不断从 YOUNG 氏矩阵中取出最小值，组成排序后的最小数组，EXTRACT_MIN 的复杂度为
$O(2n) = O(n)$，需调用 $n^2$ 次，共需用时 $O(n^3)$  
3. 两步共需耗时 $O(n^3)$ 

##### BUILD-YOUNG

In [0]:
def build_young(array):
  """将数组转换为 Young 氏矩阵"""
  size = math.ceil(math.sqrt(len(array)))
  A = [[float('inf') for i in range(size)] for j in range(size)]
  for i in array:
    young_insert(A, i)
  return A

##### YOUNG-SORT

In [0]:
def young_sort(array):
  A = build_young(array)
  for i in range(len(A)**2):
    mini = young_extract_min(A)
    if mini == float('inf'):
      break
    array[i] = mini


##### TEST

In [0]:
size = 4
array = [random.randint(1, 99) for i in range(size ** 2)]
print(array)
young_sort(array)
print(array)

[32, 36, 63, 58, 42, 60, 4, 97, 96, 90, 22, 72, 61, 88, 6, 58]
[4, 6, 22, 32, 36, 42, 58, 58, 60, 61, 63, 72, 88, 90, 96, 97]


#### f
在 $O(m+n)$ 内判断一个数是否储存在 $ m \times n$ 的 Young 氏矩阵中

基本思路：
1. 将给定的数 $num$ 与矩阵右上角的元素$Y[1,n]$相比较，如果相等，则返回 True, 如果$Y[1,n] < num$，
则可以去掉第 1 行，否则，则可以去掉第 $n$ 列
2. 递归调调用 1，直至最后一个元素，如果最后一个元素仍不满足，则返回 False
3. 可先将 $num$ 与 $Y[1,1]$ 相比较，如果 $num < Y[1,1]$， 则可以直接返回 False
第 2 步递归调用时的复杂度 $$T(p)=T(p-1)+O(1)\quad max(p)=m+n$$  
整体的时间复杂度为 $O(m+n)$

In [0]:
def find_num_in_young(A, num):
  def helper(i,j):
    if A[i][j] == num:
      return True
    if A[i][j] < num and i < len(A) - 1:
      return helper(i+1, j)
    elif j > 0:
      return helper(i, j-1)
    return False
  return False if A[0][0] > num else helper(0, len(A[0]) - 1)

In [0]:
size = 4
array = [random.randint(1, 16) for i in range(size ** 2)]
A = build_young(array)
print_matrix(A)

[ 
 3 ,  6 ,  6 , 11 , 

 7 ,  7 , 12 , 12 , 

 9 , 10 , 13 , 13 , 

10 , 11 , 14 , 14 , 

]


In [0]:
num = 0
print("find {} in A: {}".format(num, find_num_in_young(A, num)))
num = 4
print("find {} in A: {}".format(num, find_num_in_young(A, num)))
num = 14
print("find {} in A: {}".format(num, find_num_in_young(A, num)))
num = 17
print("find {} in A: {}".format(num, find_num_in_young(A, num)))

find 0 in A: False
find 4 in A: False
find 14 in A: True
find 17 in A: False
